# Best run so far: -6.96

In [2]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='ibm-q/open/main',
    token='4c490f1c7d2312d4615a8061d67227acf084aef01e4a709efffeae7692faa04b6b0e3357612c887185e8e63e9aa47b28a268b1c2f65a4921f4aaa10e8d58ee3f'
)
job = service.job('d09cqvf6rr3g00871xdg')
job_result = job.result()

for idx, pub_result in enumerate(job_result):
    print(f"Expectation values for pub {idx}: {pub_result.data.evs}")

Expectation values for pub 0: [-6.96574784]


In [4]:
import numpy as np
x_tested = np.array([ 4.0237908 , -0.13743861,  1.90743355,  1.14023283, -0.05723445,
        2.1407677 ,  6.95619161,  5.48023463,  5.09091184,  7.15576169,
        5.48894093,  6.7263609 ,  5.64666831,  4.12225883,  5.60655217,
        0.74129872,  5.41548997,  3.92536714])

In [ ]:
# %%
from functools import *
import numpy as np
import matplotlib.pyplot as plt
import scipy as sci

from defining_the_hamiltonian import fermi_hubbard_hamiltonian, swap_electron, number_up_down

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler import PassManager

from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime.fake_provider import FakeMelbourneV2
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke
from qiskit_aer import AerSimulator

from qiskit import QuantumCircuit

from qiskit.circuit.library import excitation_preserving

from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.hamiltonians import FermiHubbardModel
from qiskit_nature.second_q.hamiltonians.lattices import Lattice
import time

from qiskit_aer.primitives import EstimatorV2


# Replace 'YOUR_TOKEN_HERE' with your actual token string
token="4c490f1c7d2312d4615a8061d67227acf084aef01e4a709efffeae7692faa04b6b0e3357612c887185e8e63e9aa47b28a268b1c2f65a4921f4aaa10e8d58ee3f"
QiskitRuntimeService.save_account(channel="ibm_quantum", token=token, overwrite=True)
service = QiskitRuntimeService(channel="ibm_quantum", instance="ibm-q/open/main")
# # Choose backend
backend = service.backend("ibm_sherbrooke")  # or "ibm_brisbane", "ibm_oslo", etc.



# %%
# backend = AerSimulator()
# sherbrooke = FakeSherbrooke()
# sherbrooke.refresh(service)
# backend = AerSimulator.from_backend(sherbrooke)


# %%
verticies = 6
nodes = verticies
weighted_edges = [
    (0,1,1),(0,2,1),(0,3,1),(0,4,1),
    (5,1,1),(5,2,1),(5,3,1),(5,4,1),
    (1,2,1),(2,3,1),(3,4,1),(4,1,1)
    ]

octahedron = Lattice.from_nodes_and_edges(num_nodes=nodes, weighted_edges=weighted_edges)
lattice = octahedron.uniform_parameters(
    uniform_interaction=-1,uniform_onsite_potential=0)
fermi_hubb = FermiHubbardModel(lattice, onsite_interaction=1)

# %%
# hamiltonian = JordanWignerMapper().map(fermi_hubb.second_q_op())
hamiltonian = fermi_hubbard_hamiltonian()

qc = QuantumCircuit(verticies*2)
qc.x(2)
qc.x(verticies+2)
# qc.rxx(theta, 2,3)


# ansatz = evolved_operator_ansatz([edge_op, same_site_op], reps=1, insert_barriers=True)
# ansatz = (n_local(verticies*2, "rz", "cx", "full", reps=2, insert_barriers=True))
# entanglement = [(i+verticies*spin, j+verticies*spin) 
#                 for i,j in edges for spin in [0,1]] + [(i, i+verticies) for i in range(verticies)]
# ansatz = n_local(verticies*2, "rx", "cx", "linear" , reps=10)
six_qubit_ansatz = excitation_preserving(verticies, reps=1, 
                                          insert_barriers=True,
                                          skip_final_rotation_layer=False,
                                          entanglement = [(2,3),(1,2),(3,4),(0,1),(4,5)],
                                          mode = "iswap",
                                          parameter_prefix="A")

# qc.compose(six_qubit_ansatz,
#            qubits=(range(0, verticies)),
#            inplace=True)
# for i in range(0, verticies):
#     qc.cx(i,i+6)
qc.compose(six_qubit_ansatz,
           qubits=range(0,verticies),
           inplace=True)
qc.compose(six_qubit_ansatz,
           qubits=(range(verticies, 2*verticies)),
           inplace=True)
ansatz = qc


# ansatz.draw("mpl")

num_of_parameters = ansatz.num_parameters

print(f"There are {num_of_parameters} parameters.")
# ansatz.decompose().draw("mpl", style="iqp")

# %%

pm:PassManager = generate_preset_pass_manager(backend=backend, optimization_level=3)

ansatz_isa = pm.run(ansatz)
hamiltonian_isa = hamiltonian.apply_layout(layout=ansatz_isa.layout)
print(ansatz_isa.count_ops())

# %%
x0 = np.array([3.19092793, 1.01615098, 0.18980219, 3.04553264, 1.00397232,
       3.56617761, 7.07732137, 5.26950063, 4.13405596, 3.92659396,
       2.36595772, 5.30784715, 3.51164928, 2.01101589, 6.84437558,
       5.26728   , 3.52984988])
# len(x0)

# cost_history_dict = {
#     "prev_vector": None,
#     "iters": 0,
#     "cost_history": [],
# }

# def cost_func(params, ansatz, hamiltonian, estimator):
#     pub = (ansatz, [hamiltonian], [params])
#     result = estimator.run(pubs=[pub]).result()
#     energy = result[0].data.evs[0]

#     cost_history_dict["iters"] += 1
#     cost_history_dict["prev_vector"] = params
#     cost_history_dict["cost_history"].append(energy)
#     print(f"Iters. done: {cost_history_dict['iters']} [Current cost: {energy}]")

#     return energy

# %%
SHOTS = 10000
estimator = Estimator(backend, options={"default_shots": SHOTS})
optimal_circuit = ansatz_isa.assign_parameters(x0)
pub = (optimal_circuit, [hamiltonian_isa])
result = estimator.run([pub]).result()
energy = result[0].data.evs[0]

print(energy)

with open(f"Real-Output-{time.time()}", 'a') as f:
    print(f"Ground state energy from quantum computer: {energy}", file = f)
    print(f"result: {result}", file = f)

# %%
